## Sentiment Analysis with NLP

### Overview

Apple and Google are two leading technology companies with a vast presence in the consumer electronics and software markets. Both companies recently launched new products: Apple introduced the iPhone 15, while Google unveiled the Pixel 8. To understand consumer perceptions and brand positioning, both companies want to conduct sentiment analysis on social media to gauge public reactions to these launches.

### Objectives:

1. Compare consumer sentiment toward the Apple and Google products.
2. Identify specific emotions associated with each brand’s products.
3. Analyze customer feedback to inform future product development and marketing strategies.

## NLP Procedures
**1. Data Collection:**

. Gather tweets mentioning EcoTech and its new product line using Twitter's API. Use keywords such as "EcoTech," "biodegradable," and relevant hashtags.

**2. Preprocessing:**

. Text Cleaning: Remove URLs, mentions (@username), hashtags, and special characters from the tweet text.

. Tokenization: Split the cleaned text into individual words or tokens.

. Lowercasing: Convert all tokens to lowercase to maintain uniformity.

. Stopword Removal: Remove common stopwords (e.g., "and," "the," "is") that do not contribute to sentiment.

**3. Sentiment Analysis:**

. Use a pre-trained sentiment analysis model (like VADER or a fine-tuned BERT model) to classify each tweet as positive, negative, or neutral.
Extract and categorize the emotions (e.g., joy, anger, surprise) expressed in tweets using an emotion detection model.

**4. Feature Extraction:**

. Use techniques like TF-IDF or word embeddings (e.g., Word2Vec or GloVe) to represent tweet text as numerical features for further analysis.

**5. Analysis:**

. Aggregate Sentiment Scores: Calculate the overall sentiment score and categorize tweets by positive, negative, and neutral sentiment.

. Emotion Analysis: Assess the frequency of specific emotions associated with EcoTech, such as joy or frustration, and identify patterns in emotional responses.

**6. Visualization:**

. Create visual representations (like bar charts or word clouds) to illustrate overall sentiment, emotion distribution, and trends over time.

**7. Reporting:**

. Prepare a report summarizing findings, including key sentiments, emotional trends, and actionable insights for the marketing team. Recommendations could include addressing negative sentiment themes or amplifying positive feedback in future campaigns.

. Through this sentiment analysis, EcoTech can gain valuable insights into customer perceptions of their new product line, allowing them to make informed decisions to enhance their marketing strategy and improve overall brand reputation.

### Task
Build a model that can rate the sentiment of a Tweet based on its content.

## Data
Let us first import all the relevant modules and load the dataset.

In [1]:
# Importing th relevant libraries and modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
import os

In [3]:
import re
import nltk
import string
from nltk import FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# import spacy
# # nlp = spacy.load("en_core_web_sm")

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm

In [6]:
# Check GPU status
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [7]:
# Downloading nltk metrics
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madservices/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/madservices/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/madservices/nltk_data...


True

In [11]:
# Loading the dataset
df = pd.read_csv("data/judge-1377884607_tweet_product_company.csv", encoding='unicode_escape')

# Checking the dataset
df.head()
# print()
# print(df.describe())

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


The dataset consists of 9,093 tweets organized into three columns: tweet_text, emotion_in_tweet_is_directed_at, and is_there_an_emotion_directed_at_a_brand_or_product. The tweet_text column contains the tweet content with one missing entry, while the emotion_in_tweet_is_directed_at column has 3,291 non-null values, indicating significant missing data regarding the emotional context directed at individuals or brands. The third column is complete, capturing whether an emotion is directed at a brand or product. All columns are of object data type, suggesting that further processing may be required for analysis. This dataset is valuable for sentiment analysis and market research, offering insights into customer emotions and brand perceptions on social media. However, the high amount of missing values in the emotion-related column necessitates careful consideration in analysis.